# ADZD L4 - Strumieniowanie

## Autor: Paweł Mendroch

### Setup

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession\
    .builder\
    .config("spark.sql.streaming.schemaInference", True)\
    .getOrCreate()

In [39]:
import sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, window, sum, avg, col, to_timestamp
from pyspark.sql.types import StructType, TimestampType
from pyspark.sql.streaming import DataStreamReader

In [3]:
openaqSchema = StructType()\
              .add("city", "string")\
              .add("value", "double")\
              .add("local", TimestampType())

In [4]:
stream_pm25 = spark\
                .readStream\
                .format("sqs")\
                .option("queueUrl", "<queueUrl>")\
                .option("accessKey", "<accessKey>")\
                .option("secretKey", "<secretKey>")\
                .option("region", "us-west-1")\
                .load()

In [6]:
stream_pm10 = spark\
                .readStream\
                .format("sqs")\
                .option("queueUrl", "<queueUrl>")\
                .option("accessKey", "<accessKey>")\
                .option("secretKey", "<secretKey>")\
                .option("region", "us-west-1")\
                .load()

### Zadanie 1

In [74]:
query_avg_pm25.stop()

In [73]:
query_avg_pm25 = stream_pm25\
                    .select("city", "value", "date.local")\
                    .withColumn("timestamp", col("local").cast(TimestampType()))\
                    .groupBy(window("timestamp", "30 minutes", "15 minutes"), "city")\
                    .agg(avg("value").alias("AVG pm25"))\
                    .writeStream\
                    .outputMode("complete")\
                    .format("console")\
                    .option("truncate", "false")\
                    .trigger(processingTime="10 seconds")\
                    .start()

# query_avg_pm25 = stream_pm25\
#                     .select("city", "value", "date.local")\
#                     .withColumn("timestamp", col("local").cast(TimestampType()))\
#                     .withWatermark("timestamp", "15 minutes")\
#                     .groupBy(window("timestamp", "30 minutes", "15 minutes"), "city")\
#                     .agg(avg("value").alias("AVG pm25"))\
#                     .writeStream\
#                     .outputMode("append")\
#                     .format("console")\
#                     .option("truncate", "false")\
#                     .trigger(processingTime="10 seconds")\
#                     .start()

# query_avg_pm25.awaitTermination()

### Zadanie 2

In [9]:
monitoring_dir = "monitor"
dataset_dir = "data"

In [10]:
stream_from_csv = spark\
                    .readStream\
                    .schema(openaqSchema)\
                    .csv(monitoring_dir)

In [12]:
query_avg_pm25_to_csv.stop()

In [11]:
query_avg_pm25_to_csv = stream_pm25\
                            .select("city", "value", "date.local")\
                            .writeStream\
                            .format("CSV")\
                            .outputMode("append")\
                            .option("checkpointLocation", "applicationHistory")\
                            .option("path", dataset_dir)\
                            .option("truncate", "false")\
                            .trigger(processingTime="10 seconds")\
                            .start()

# query_avg_pm25_to_csv.awaitTermination()

In [100]:
query_watermark_avg_pm25.stop()

In [101]:
query_watermark_avg_pm25 = stream_from_csv\
                                .select("city", "value", "local")\
                                .withColumn("timestamp", col("local").cast(TimestampType()))\
                                .withWatermark("timestamp", "7 days")\
                                .groupBy(window("timestamp", "60 minutes"), "city")\
                                .agg(avg("value").alias("AVG pm25"))\
                                .writeStream\
                                .outputMode("update")\
                                .format("console")\
                                .option("truncate", "false")\
                                .trigger(processingTime="10 seconds")\
                                .start()

# query_avg_pm25.awaitTermination()

### Zadanie 3

In [85]:
substream_pm25 = stream_pm25\
                    .withColumn("timestamp", col("date.local").cast(TimestampType()))\
                    .select("city", "value", "timestamp", window("timestamp", "30 minutes", "15 minutes"))\
                    .withWatermark("timestamp", "15 minutes")

substream_pm10 = stream_pm10\
                    .withColumn("timestamp", col("date.local").cast(TimestampType()))\
                    .select("city", "value", "timestamp", window("timestamp", "30 minutes", "15 minutes"))\
                    .withWatermark("timestamp", "15 minutes")

# .withWatermark("timestamp", "10 minutes")

In [94]:
joined_substreams = substream_pm25.alias("s1")\
                        .join(substream_pm10.alias("s2"), (col("s1.window") == col("s2.window")) & (col("s1.city") == col("s2.city")))\
                        .withWatermark("s1.timestamp", "15 minutes")\
                        .withWatermark("s2.timestamp", "15 minutes")

In [95]:
query_joined_substreams.stop()

NameError: name 'query_joined_substreams' is not defined

In [98]:
query_joined_substreams = joined_substreams\
                            .select("s1.city", "s1.value", "s2.value", "s1.timestamp", "s2.timestamp")\
                            .withWatermark("s1.timestamp", "15 minutes")\
                            .withWatermark("s2.timestamp", "15 minutes")\
                            .groupBy(window("s1.timestamp", "30 minutes", "15 minutes"), "s1.city")\
                            .agg(avg("s1.value").alias("AVG pm25"), avg("s2.value").alias("AVG pm10"))\
                            .writeStream\
                            .outputMode("append")\
                            .format("console")\
                            .option("truncate", "false")\
                            .trigger(processingTime="10 seconds")\
                            .start()

# query_avg_both_substreams.awaitTermination()

AnalysisException: Append output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark;;
Aggregate [window#7774, city#6], [window#7774 AS window#7764, city#6, avg(value#2) AS AVG pm25#7771, avg(value#20) AS AVG pm10#7773]
+- Filter ((timestamp#7546 >= window#7774.start) AND (timestamp#7546 < window#7774.end))
   +- Expand [ArrayBuffer(named_struct(start, precisetimestampconversion(((((CASE WHEN (cast(CEIL((cast((precisetimestampconversion(timestamp#7546, TimestampType, LongType) - 0) as double) / cast(900000000 as double))) as double) = (cast((precisetimestampconversion(timestamp#7546, TimestampType, LongType) - 0) as double) / cast(900000000 as double))) THEN (CEIL((cast((precisetimestampconversion(timestamp#7546, TimestampType, LongType) - 0) as double) / cast(900000000 as double))) + cast(1 as bigint)) ELSE CEIL((cast((precisetimestampconversion(timestamp#7546, TimestampType, LongType) - 0) as double) / cast(900000000 as double))